In [1]:
import os
import feedparser
import requests
import subprocess
import assemblyai as aai
import threading

## Configuring Variables


In [6]:
# Podcast RSS Feed URLs
nasil_olunur_url = "https://feeds.simplecast.com/TRRbx_5C"
ybyk_url = "https://feeds.simplecast.com/leyT8Ibc"

# File Directory Variables
download_dir = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/episodes_nasil_olunur"
transcription_folder = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes"

# Track Downloaded Episodes
downloaded_episodes = set()


## Downloading Podcast Episodes

In [3]:
def download_and_compress(title, url):
    """
    Downloads a compressed mp3 version of a podcast episode
    :param title: Episode title
    :param url: The specific episode url from RSS feed
    :return: null
    """

    # clear the filename
    title = title.replace("- ", "-").replace(" ", "_").replace("/", "-")
    filename = os.path.join(download_dir, f"{title}.mp3")
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    ffmpeg_command = [
        "ffmpeg", "-i", url,                # Input URL
        "-b:a", "64k",                      # Set audio bitrate
        "-f", "mp3",                        # Output format
        "-y",                               # Overwrite existing file
        "-loglevel", "error",               # Suppress all but error output
        filename                            # Output file path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Downloaded and compressed: {title}")
    except subprocess.CalledProcessError:
        print(f"Failed to download and compress: {title}")

In [4]:
def check_new_episodes(url, begin,end):
    """
    Checks RSS feed for new episodes
    :param url: Podcast's RSS feed url
    :param begin: start checking from this episode no
    :param end: end checking at this episode no
    :return: none
    """
    feed = feedparser.parse(url)

    for episode in feed.entries[begin:end]:
        title = episode["title"]
        audio_url = episode.enclosures[0].href
        print(f"Title: {title} | Audio Url: {audio_url}")
        if title not in downloaded_episodes:
            downloaded_episodes.add(title)
            download_and_compress(title, audio_url)

In [11]:
feed = feedparser.parse(nasil_olunur_url)

start_transcription_from = 235
e = len(feed.entries) - start_transcription_from
# s = e - 15 # how many more episodes to transcribe
print({start_transcription_from}, {e})

# check_new_episodes(nasil_olunur_url, s, e)

{235} {8}


## Transcription with Assembly AI

In [15]:
aai.settings.api_key = "ASSEMBLY_API_KEY"

config = aai.TranscriptionConfig(
    speaker_labels=True,
    speakers_expected=2,
    language_code="tr",
    entity_detection=True #entity detection ON
)

# Simdilik sadece Nasil Olunur icin yaptim
config.set_custom_spelling(
  {
    "Nilay": ["Nile"],
  }
)

episode_dict = {}

In [16]:
def transcribeAudio(audio_file):
    """
    Transcribes single audio file (mp3)
    :param audio_file: audio file to be transcripted
    """

    ep_no = audio_file[0:3]
    guest = audio_file[4:-4]

    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(os.path.join(download_dir, audio_file))

    episode_dict[audio_file] = [audio_file, ep_no, guest, transcript.id]

In [66]:
def transcribeAudioFromURL(audio_url):
    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(audio_url)

    return transcript.text[0:1000]

# transcribeAudioFromURL(audio_url)

id = "7f31f327-581c-40d7-b95c-2cb99669aceb"
transcript = aai.Transcript.get_by_id(id)

print(transcript.text[0:1000])



Merhabalar, ben Nilay Yörnek. Yeni bir şeyleri öğrenmeye merakla herkesin ilgisinin çekeceğini düşündüğüm... ...Nasıl Olunur adlı podcast serimi hoş geldiniz. Bu seride mikrofonu mesleklerini ustalıkla icra ettiğini düşündüğüm insanlara yöneldim. Onlara aynı soruyu soruyorum. Nasıl olunur? Bu bölümde konuğum bir bağlama üstadı... ...Türk Halk Müziği sanatçısı, eğitimci Erdal Erzincan. Hoş geldiniz. Hoş buldum. Box Müzesi'nin Anadolu ödülleri çift yıllarda kurumlara, tek yıllarda kişilere veriliyor. Yeni'yi Aramak üst başlığıyla verilen 2024 müzik ödülünü de Erdal Erzincan Gezici Bağlama Atölyesi aldı. Tebrik ediyorum. Çok teşekkür ederim. 2019'Dan bu yana ödüllerin verilmesi için baya baya çalıştığına şahit oldum. Hüsamettin Koçer'e de söyledim. Hemen akşam konuşuyoruz. Çok havalıydı ödül töreni dedim hocam. Sizin ödülünüzü Bülent Ortaçgil açıkladı. Ekrem İmamoğlu'ndan aldınız ödülü. Baya havalıydı. Başka ödül törenlerinde hocam dedim bir kişi üç ödül birden veriyor sizde dedim iki kiş

In [59]:
def batchTranscription():
    """
    Transcribes multiple audio files concurrently
    """
    threads = []

    for filename in os.listdir(download_dir):
        if int(filename[0:3]) > 158:
            thread = threading.Thread(target=transcribeAudio, args=(filename,))
            threads.append(thread)
            thread.start()

    for thread in threads:
        thread.join()

    print("All transcriptions are complete.")

In [60]:
batchTranscription()

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection.py", line 103, in handle_request
    return self._connection.handle_request(request)
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/http11.py", line 136, in handle_request
    raise exc
  File "/

KeyboardInterrupt: 

### Retrieve Transcript by ID

In [101]:
def get_transcript():

  for key, value in episode_dict.items():
      transcript_id = value[3]
      transcript = aai.Transcript.get_by_id(transcript_id)
      # print(transcript.text[300:400])


In [62]:
t_id = "cce21de0-04df-4e6d-a1db-e283bcefd88d"
transcript = aai.Transcript.get_by_id(t_id)
print(transcript.text[0:1000])

Merhabalar, ben Nilay Örnek. Yeni bir şeyler öğrenmeye merakla herkesin ilgisini çekeceğini düşündüğüm Nasıl Olunur adlı podcast serimi hoş geldiniz. Bu seride mikrofonu mesleklerini ustalıkla icra ettiğini düşündüğüm insanlara yöneltip onlara aynı soruyu soruyorum. Nasıl olunur? Bu bölümde konuğum, profesyonel bilardo oyuncusu, Türkiye'yi dünyada temsil etmiş, dünya şampiyonu olmuş bir isim Semih Saygıner. Hoş geldiniz. Hoş bulduk. Çok memnunum geldiğiniz için. Teşekkür ederim. Şimdi Semih Bey malum, Türkiye'de Bilardo'nun imajını değiştiren, saygınlığını oturtan, Bilardo'ya ilgiyi arttıran bir isim. Lenk lenk vardır Çinli piyanist hep onun yüzlerce, binlerce kişiyi, çocukları özellikle piyanoya teşvik ettiği söylenir Çin'de. Semih Bey'in de Bilardo'da öyle bir etkisi olduğu malum. Tam bir nasıl olunur? İnsanı biraz açmak istiyorum izniniz olursa. Tabii ki. Zaten programda siz de dinleyeceksiniz, göreceksiniz şahane bir anlatıcı. Hem sadece bilardo oyuncusu, bir sporcu değil, müzisyen

## Transcript -> txt file

In [9]:
def write_text_to_file(transcript, title, guest):
    file_path = "/transcripted_episodes/txt_files"
    guest = guest.replace("_", " ")

    text = ""
    for utterance in transcript.utterances:
        if utterance.speaker == "A":
          speaker = "Nilay Örnek"
        elif utterance.speaker == "B":
          speaker = guest

        text += f"{speaker}: {utterance.text}\n"

    with open(f"{file_path}/{title}.txt", "w") as f:
        f.write(text)

In [10]:
# Create txt Files
for key,value in episode_dict.items():
    title = key[:-4]
    guest = value[2]

    transcript_id = value[3]
    transcript = aai.Transcript.get_by_id(transcript_id)

    write_text_to_file(transcript, title, guest)

NameError: name 'episode_dict' is not defined

## Transcript -> .doc file

In [17]:
from docx import Document

new_transcription_folder = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/episodes_with_timestamps"
def write_to_word(transcript, title, guest, ep_no):
  file_path = os.path.join(new_transcription_folder, f"{title}.docx")
  guest = guest.replace("_", " ")

  doc = Document()
  doc.add_heading(f"Nasıl Olunur {ep_no}. Bölüm - {guest}", level=0)

  def time_conversion(millisec):
      second = (millisec // 1000) % 60
      minute = (millisec // 60000) % 60
      hour = millisec // 3600000

      return f"[{hour:02}:{minute:02}:{second:02}]"

  for utterance in transcript.utterances:
    if utterance.speaker == "A":
      speaker = "Nilay Örnek"
    elif utterance.speaker == "B":
      speaker = guest

    time = time_conversion(utterance.start)

    paragraph = doc.add_paragraph()
    paragraph.add_run(f"{speaker} {time}: ").bold = True
    paragraph.add_run(utterance.text)


  doc.save(file_path)

In [8]:
# Create Word Files
for key,value in episode_dict.items():
    title = key[:-4]
    guest = value[2]
    episode_no = value[1]

    transcript_id = value[3]
    transcript = aai.Transcript.get_by_id(transcript_id)

    write_to_word(transcript, title, guest, episode_no)